# Example of using the Borehole3D class

## Imports

In [1]:
from striplog import Component, Decor, Legend, Lexicon, Striplog
from core.omf import Borehole3D, striplog_legend_to_omf_legend 
from utils.lexicon.lexicon_fr import lexicon_fr, legend_fr
from utils.io import boreholes_from_files, gdf_viewer
from core.core import Project
from core.orm import BoreholeOrm, PositionOrm, Base 

import matplotlib.pyplot as plt
from os import remove
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from definitions import ROOT_DIR

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/striplog/legend.py:687: UserWarning: This legend contains duplicate components.
  warnings.warn(w)
/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/striplog/legend.py:687: UserWarning: This legend contains duplicate components.
  warnings.warn(w)


## Reading of CSV files in a dir

In [2]:
save_dir = ROOT_DIR+'/CF_data/Donnees_fusionnees/'

In [3]:
df = pd.read_csv(save_dir+'Pilote_test.csv', sep=',')

In [4]:
gdf_viewer(df)

Rows : 86, columns : 10


interactive(children=(IntSlider(value=10, description='rows', max=86, min=10, readout=False), IntSlider(value=…

## Data Checking

In [5]:
df.drop(index=df.query('X!=X').index, inplace=True)
df.drop(index=df.query('Description!=Description').index, inplace=True)

In [6]:
df.query('ID=="201"')

,ID,Ep_litho,X,Y,Z,Long_for,Litho_base,Litho_top,Diam_for,Description
0,201,0.6,152890.245758,122596.47426,101.926886,2.4,1.2,0.6,75.0,Remblais avec briques de construction couleur ...
1,201,1.2,152890.245758,122596.47426,101.926886,2.4,2.4,1.2,75.0,Briquaille de déchets de construction et galet...


In [7]:
df['Ep_litho']=df['Litho_base']-df['Litho_top']

## Create Boreholes from dataframes or files

In [8]:
bhs_df=[df] # list of dataframes

In [9]:
boreholes, components, link_intv_comp = boreholes_from_files(bhs_df, verbose=True,
                                                litho_field='Description',
                                                lexicon=lexicon_fr,
                                                diam_field='Diam_for', 
                                                litho_top_field='Litho_top',
                                                litho_base_field='Litho_base',
                                                thick_field='Ep_litho')


Dataframe 0 processing...
|__ID:'201'
Summary : [Striplog(2 Intervals, start=0.6, stop=2.4)]
{0: {'description': 'Remblais avec briques de construction couleur orange, gris, verdâtre  ', 'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7f2cbcc30dc0>, 'base': <core.orm.PositionOrm object at 0x7f2cbcc30880>}, 1: {'description': "Briquaille de déchets de construction et galets roulés - présence d'eau - légère odeur de naphtalène - Bloqué à 2,4 m sur caillasse (et béton ?)", 'interval_number': 1, 'top': <core.orm.PositionOrm object at 0x7f2cbcc3ee20>, 'base': <core.orm.PositionOrm object at 0x7f2cbcc3ed60>}}

|__ID:'205'
Error : No lithology matching with 'Couche de boue en surface liée à l'intervention de Géosonda' in given lexicon
Summary : [Striplog(7 Intervals, start=0.0, stop=4.8)]
{2: {'description': 'white sand', 'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7f2cbcc3eca0>, 'base': <core.orm.PositionOrm object at 0x7f2cbcc3e460>}, 3: {'description': 'L

/home/yanathan/Projects/GSDMA/utils/io.py:478: SAWarning: relationship 'ComponentOrm.intervals' will copy column Components.id to column Linkintervalcomponent.comp_id, which conflicts with relationship(s): 'IntervalOrm.components' (copies Components.id to Linkintervalcomponent.comp_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   The 'overlaps' parameter may be used to remove this warning.
  boreholes.append(BoreholeOrm(id=bh_name))
/home/yanathan/Projects/GSDMA/utils/io.py:478: SAWarning: relationship 'ComponentOrm.intervals' will copy column Intervals.id to column Linkintervalcomponent.int_id, which conflicts with relationship(s): 'IntervalOrm.components' (copies Intervals.id to

In [10]:
litho_list = []
color_list = []
for i, c in components.items():
    if not hasattr(c, 'lithology'):
        c.lithology = 'sable'
        c.colour = 'rouge'
    elif c.lithology == 'sand':
        c.lithology = 'sable'
        c.colour = 'blanc'
    if c.lithology not in litho_list:
        litho_list.append(c.lithology)
    if hasattr(c, 'colour') and c.colour not in color_list:
        color_list.append(c.colour)

In [11]:
litho_list, color_list

(['sable', 'remblais', 'limon', 'limons', 'schistes'],
 ['rouge',
  'orange gris',
  'verdâtre',
  'gris',
  'blanc',
  'brun',
  'noire',
  'brique',
  'sable beige',
  'brun beige',
  'noir verdâtre',
  'jaune',
  'noir',
  'verdâtre brun'])

In [12]:
components[9]

grainsize,grossiers
lithology,sable
colour,rouge


In [ ]:
memoris_components = [Component({'lithology': 'remblais'}),
              Component({'lithology': 'limon'}),
              Component({'lithology': 'schistes'}),
              Component({'lithology': 'sable'}),
              Component({'lithology': 'anthropique'})
              ]
list_of_decors=[]
hatches = ['=', 'x', '.', 's', '=']
colours = ['#888888', '#882222', '#AAAAAA', '#CC22CC', '#CC2222']

for i in range(len(memoris_components)):
    if hasattr(memoris_components[i], 'colour'):
        c = memoris_components[i].colour
    else:
        c=colours[i]
    
    if hasattr(memoris_components[i], 'hatch'):
        h = memoris_components[i].hatch
    else:
        h=hatches[i]
        
    d = {'color': c,
         'hatch': h,
         'component': memoris_components[i],
         'width': 3}
    decor = Decor(d)
    list_of_decors.append(decor)
    
memoris_legend = Legend(list_of_decors)

In [13]:
boreholes

[<core.orm.BoreholeOrm>(Name=201, Length=1.7999999999999998, Diameter=75.0, Intervals=2),
 <core.orm.BoreholeOrm>(Name=205, Length=4.8, Diameter=75.0, Intervals=7),
 <core.orm.BoreholeOrm>(Name=208, Length=4.8, Diameter=75.0, Intervals=5),
 <core.orm.BoreholeOrm>(Name=212, Length=4.8, Diameter=75.0, Intervals=5),
 <core.orm.BoreholeOrm>(Name=207, Length=4.8, Diameter=75.0, Intervals=7),
 <core.orm.BoreholeOrm>(Name=214, Length=4.8, Diameter=75.0, Intervals=7),
 <core.orm.BoreholeOrm>(Name=217, Length=4.2, Diameter=75.0, Intervals=5),
 <core.orm.BoreholeOrm>(Name=221, Length=1.4, Diameter=75.0, Intervals=2),
 <core.orm.BoreholeOrm>(Name=223, Length=1.3, Diameter=75.0, Intervals=1),
 <core.orm.BoreholeOrm>(Name=225, Length=4.8, Diameter=75.0, Intervals=7),
 <core.orm.BoreholeOrm>(Name=224, Length=2.4, Diameter=75.0, Intervals=2),
 <core.orm.BoreholeOrm>(Name=219, Length=1.5, Diameter=75.0, Intervals=2),
 <core.orm.BoreholeOrm>(Name=220, Length=0.5, Diameter=75.0, Intervals=1),
 <core.orm

In [14]:
components

{0: Component({'material': 'béton', 'grainsize': 'galets', 'Pollutant': 'naphtalène', 'lithology': 'sable', 'colour': 'rouge'}),
 1: Component({'lithology': 'remblais', 'colour': 'orange gris'}),
 2: Component({'lithology': 'remblais', 'Pollutant': 'naphtalène'}),
 3: Component({'lithology': 'limon', 'grainsize': 'grossiers', 'colour': 'verdâtre'}),
 4: Component({'lithology': 'remblais', 'grainsize': 'grossiers', 'Pollutant': 'naphtalène'}),
 5: Component({'lithology': 'remblais', 'colour': 'gris'}),
 6: Component({'modifier': 'sableux', 'colour': 'rouge', 'lithology': 'sable'}),
 7: Component({'lithology': 'sable', 'colour': 'blanc'}),
 8: Component({'lithology': 'limon', 'colour': 'brun'}),
 9: Component({'grainsize': 'grossiers', 'lithology': 'sable', 'colour': 'rouge'}),
 10: Component({'lithology': 'remblais', 'modifier': 'sableuse', 'colour': 'brun'}),
 11: Component({'lithology': 'limons', 'colour': 'verdâtre'}),
 12: Component({'lithology': 'remblais', 'grainsize': 'grossiers'

In [15]:
len(boreholes), len(components)

(15, 48)

## Create a project and save data in a database

In [16]:
remove('tmp_files/tfe_orm_db.db')

In [17]:
engine = create_engine('sqlite:///tmp_files/tfe_orm_db.db', echo=True)

In [18]:
Base.metadata.create_all(engine)

2021-06-18 03:31:30,847 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-18 03:31:30,848 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Boreholes")
2021-06-18 03:31:30,848 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-18 03:31:30,850 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Boreholes")
2021-06-18 03:31:30,850 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-18 03:31:30,851 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Positions")
2021-06-18 03:31:30,851 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-18 03:31:30,852 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Positions")
2021-06-18 03:31:30,852 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-18 03:31:30,854 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Intervals")
2021-06-18 03:31:30,854 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-18 03:31:30,855 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Intervals")
2021-06-18 03:31:30,856 INFO sqlalchemy.engine.Engine 

In [19]:
Session = sessionmaker(bind=engine)
session = Session()

In [20]:
p = Project(session, legend=legend_fr, lexicon=lexicon_fr)
p.add_components(components)

2021-06-18 03:31:30,902 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-18 03:31:30,904 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-06-18 03:31:30,905 INFO sqlalchemy.engine.Engine [generated in 0.00089s] ()
2021-06-18 03:31:30,910 INFO sqlalchemy.engine.Engine INSERT INTO "Components" (id, description) VALUES (?, ?)
2021-06-18 03:31:30,910 INFO sqlalchemy.engine.Engine [generated in 0.00065s] ((0, 'Béton, galets, naphtalène, sable, rouge'), (1, 'Remblais, orange gris'), (2, 'Remblais, naphtalène'), (3, 'Limon, grossiers, verdâtre'), (4, 'Remblais, grossiers, naphtalène'), (5, 'Remblais, gris'), (6, 'Sableux, rouge, sable'), (7, 'Sable, blanc')  ... displaying 10 of 48 total bound parameter sets ...  (46, 'Limon, argileux, verdâtre'), (47, 'Limon, argileux, brun'))
2021-06-18 03:31:30,912 INFO sqlalchemy.engine.Engine COMMIT
2021-06-18 03:31:30

Add boreholes into the project

In [21]:
from striplog import Lexicon, Legend, Component
import re

In [22]:
litho = 'loess'
# litho = 'tectonite mélangés'
#test = ['Tectonite(?:s)?','Tectonite(?:s)? mélangé(?:s)?','schiste(?:s)?','Cataclasite(?:s)?']
test = [l.component.lithology for l in legend_fr]

reg = list(map(lambda x: re.compile("^{:s}$".format(x), flags=re.I).search(litho), test))
r = [l.string for l in reg if l is not None]
if len(r) > 0:
    print(f"{r[0]}, {len(r)}, {litho}")

loess, 1, loess


In [23]:
legend_fr

Legend(Decor({'_colour': '#ffffe9', 'width': None, 'component': Component({'lithology': 'matériau(?:x)? meuble(?:s)?'}), 'hatch': None})
Decor({'_colour': '#ffffd5', 'width': None, 'component': Component({'lithology': 'alluvion(?:s)?'}), 'hatch': None})
Decor({'_colour': '#d3b798', 'width': None, 'component': Component({'lithology': 'remblai(?:s)?'}), 'hatch': None})
Decor({'_colour': '#f5e1bd', 'width': None, 'component': Component({'lithology': 'lœss'}), 'hatch': None})
Decor({'_colour': '#f5e1bd', 'width': None, 'component': Component({'lithology': 'loess'}), 'hatch': None})
Decor({'_colour': '#d6c59e', 'width': None, 'component': Component({'lithology': 'cendre(?:s)? volcanique(?:s)?'}), 'hatch': None})
Decor({'_colour': '#e1e3c3', 'width': None, 'component': Component({'lithology': 'colluvion(?:s)?'}), 'hatch': None})
Decor({'_colour': '#d3ca9f', 'width': None, 'component': Component({'lithology': 'lahar(?:s)?'}), 'hatch': None})
Decor({'_colour': '#ffeebf', 'width': None, 'component': Component({'lithology': 'moraine(?:s)?'}), 'hatch': None})
Decor({'_colour': '#ffcc99', 'width': None, 'component': Component({'lithology': 'tourbe(?:s)?'}), 'hatch': None})
Decor({'_colour': '#ffeaa7', 'width': None, 'component': Component({'lithology': 'gypse(?:s)?'}), 'hatch': None})
Decor({'_colour': '#ffdb67', 'width': None, 'component': Component({'lithology': 'houille(?:s)?'}), 'hatch': None})
Decor({'_colour': '#ffd345', 'width': None, 'component': Component({'lithology': 'limon(?:s)?'}), 'hatch': None})
Decor({'_colour': '#ffcb23', 'width': None, 'component': Component({'lithology': 'sable(?:s)?'}), 'hatch': None})
Decor({'_colour': '#ecb400', 'width': None, 'component': Component({'lithology': 'gravier(?:s)?'}), 'hatch': None})
Decor({'_colour': '#cfefdf', 'width': None, 'component': Component({'lithology': 'roche(?:s)? sédimentaire(?:s)?'}), 'hatch': None})
Decor({'_colour': '#d9fdd3', 'width': None, 'component': Component({'lithology': 'reoche(?:s)? clastique(?:s)?'}), 'hatch': None})
Decor({'_colour': '#ace4c8', 'width': None, 'component': Component({'lithology': 'silt(?:s)?'}), 'hatch': None})
Decor({'_colour': '#d5e6cc', 'width': None, 'component': Component({'lithology': 'argile(?:s)?'}), 'hatch': None})
Decor({'_colour': '#92dcb7', 'width': None, 'component': Component({'lithology': 'bentonite(?:s)?'}), 'hatch': None})
Decor({'_colour': '#c0d0c0', 'width': None, 'component': Component({'lithology': 'diatomite(?:s)?'}), 'hatch': None})
Decor({'_colour': '#dbfebc', 'width': None, 'component': Component({'lithology': 'molasse(?:s)?'}), 'hatch': None})
Decor({'_colour': '#bbffdd', 'width': None, 'component': Component({'lithology': 'schiste(?:s)?'}), 'hatch': None})
Decor({'_colour': '#95ffca', 'width': None, 'component': Component({'lithology': 'argilite(?:s)?'}), 'hatch': None})
Decor({'_colour': '#d6fe9a', 'width': None, 'component': Component({'lithology': 'siltite(?:s)?'}), 'hatch': None})
Decor({'_colour': '#e1f0d8', 'width': None, 'component': Component({'lithology': 'tuffeau(?:s)?'}), 'hatch': None})
Decor({'_colour': '#cdffd9', 'width': None, 'component': Component({'lithology': 'grès'}), 'hatch': None})
Decor({'_colour': '#cbefce', 'width': None, 'component': Component({'lithology': 'arénite(?:s)?'}), 'hatch': None})
Decor({'_colour': '#a6fcaa', 'width': None, 'component': Component({'lithology': 'orthoquartzite(?:s)?'}), 'hatch': None})
Decor({'_colour': '#7dffe3', 'width': None, 'component': Component({'lithology': 'calcarénite(?:s)?'}), 'hatch': None})
Decor({'_colour': '#b8eac3', 'width': None, 'component': Component({'lithology': 'arkose(?:s)?'}), 'hatch': None})
Decor({'_colour': '#bddbf1', 'width': None, 'component': Component({'lithology': 'wacke(?:s)?'}), 'hatch': None})
Decor({'_colour': '#69cf9c', 'width': None, 'component': Component({'lithology': 'silex'}), 'hatch': None})
Decor({'_colour': '#90a565', 'width': None, 'component': Component({'lithology': 'radiolarite(?:s)?'}), 'hatch': None})
Decor({'_colou

In [24]:
import matplotlib.colors as mcolors

def create_legend(components, legend, hatches=None, colors=None, width=3):
    
    list_of_decors, hatches_used = [], []
    def_hatches = ['+', 'x', '.', 's', '*', 'b', 'c', 'v', '/', 't']
    def_colors = [l.colour for l in Legend.default()] + list(mcolors.CSS4_COLORS.values())
    i=0
    
    for comp in components:
        if hasattr(comp, 'lithology'):
            comp_litho = comp.lithology
            for leg in legend:
                leg_litho = leg.component.lithology
                
                reg = re.compile("^{:s}$".format(leg_litho), flags=re.I).match(comp_litho)
                print(reg)
                if reg:
                    print(i, comp, reg)
        i += 1         

In [25]:
create_legend(components, legend_fr)

In [26]:
from utils.lexicon.lexicon_fr import COLOURS_FR

In [27]:
name = 'gris'
COLOURS_FR[name.lower()]

KeyError: 'gris'

In [ ]:
for bh in boreholes:
    p.add_borehole(bh)

In [ ]:
p.add_link_between_components_and_intervals(link_intv_comp)

In [ ]:
p.commit()
print('Boreholes in the project : ', len(p.boreholes))

In [ ]:
p.boreholes

In [ ]:
p.refresh(verbose=True)

In [ ]:
session.close()

# 3D Display

## Display one object

In [ ]:
bh = p.boreholes_3d
#print(bh.name, '\n', bh.intervals) # problem with top and base position --> kind of overlaying

In [ ]:
p.boreholes_3d[0].plot2d()

In [ ]:
p.boreholes_3d[0].plot3d()

In [ ]:
p.boreholes_3d[0].plot3d(x3d=True)

### Display project boreholes

In [ ]:
p.plot3d(labels_size=15, bg_color=["royalblue", "aliceblue"], window_size=(1300, 1000))

### Boreholes in 3D using X3D for interaction within the notebook